In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from DBSCANSupport import DBSCANSupport
from LocalPath import LOCALPATH
from sklearn.cluster import DBSCAN

In [2]:
grav = pd.read_csv(LOCALPATH + 'data/test_grav.csv').drop(columns=["old_ind"])
grav = grav[['Longitude', 'Latitude', 'Intensity']]

In [3]:
Z = DBSCANSupport.formatData(grav, 'Intensity')
#Z = Z[(Z[:, 2] > -20) & (Z[:, 2] < 7)]
data = Z

/Users/seamount_picking_stable/seamount_picking/code/_SeamountSupport.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["TrueSeamount"] = np.zeros(data.shape[0])


In [4]:
## Elbow test
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
neighbors = NearestNeighbors(n_neighbors=20)
neighbors_fit = neighbors.fit(data)
distances, indices = neighbors_fit.kneighbors(data)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
elbow = px.scatter(x=range(0, len(distances)), y=distances)#.show()

In [5]:
test_eps = np.linspace(0.1, 0.25, 20)
test_samp = np.arange(2, 7)
DBModel_test = DBSCANSupport(LOCALPATH+"data/sample_mask.txt.xlsx", train_zone=(-6, -1.5, -98, -90))
DBModel_test.addTrainingData(data)

In [ ]:
score, params, data_out  = DBModel_test.gridSearch(test_eps, test_samp, verbose=True)

In [ ]:
dfout = pd.DataFrame(data_out, columns=["Easting", "Northing", "Label", "Intensity"])
df_labeled = dfout[dfout['Label'] == 1]

In [ ]:
fig = px.scatter(df_labeled, x="Easting", y="Northing", color="Label")
fig.show()

In [ ]:
fig2 = px.scatter(dfout, x="Easting", y="Northing", color="True_Seamount")
fig2.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
  )

In [ ]:
with open(LOCALPATH + "out/best.txt", "w") as f:
    f.write("Score: " + str(score) + "\n")
    f.write("Params: " + str(params) + "\n")
DBModel_test.matchPoints().to_csv(LOCALPATH + "out/matched.csv", index=False)
dfout.to_csv(LOCALPATH + "out/labels.csv", index=False)
